In [13]:
pip install googletrans==4.0.0-rc1 pykakasi pypinyin hangul-romanize


Note: you may need to restart the kernel to use updated packages.


In [17]:
from tkinter import *
from tkinter import ttk
from googletrans import Translator, LANGUAGES
import pykakasi
from pypinyin import lazy_pinyin
from hangul_romanize import Transliter
from hangul_romanize.rule import academic

translator = Translator()
kks = pykakasi.kakasi()
transliter = Transliter(academic)

def romanize_text(text, lang_code):
    if lang_code == 'ja':  # Japanese
        result = kks.convert(text)
        return ' '.join([item['hepburn'] for item in result])
    elif lang_code in ['zh-cn', 'zh-tw']:  # Chinese
        return ' '.join(lazy_pinyin(text))
    elif lang_code == 'ko':  # Korean
        return transliter.translit(text)
    else:
        return None  # Other languages not supported

def translate_text():
    input_text = text_input.get("1.0", END).strip()
    lang = lang_choice.get()
    lang_code_list = [k for k, v in LANGUAGES.items() if v == lang.lower()]
    if input_text and lang_code_list:
        lang_code = lang_code_list[0]
        translated = translator.translate(input_text, dest=lang_code)
        text_output.delete("1.0", END)
        text_output.insert(END, translated.text)

        romanized = romanize_text(translated.text, lang_code)
        if romanized:
            text_output.insert(END, f"\n\nPronunciation: {romanized}")

root = Tk()
root.title("Real-Time Translator with Pronunciation")
root.geometry("600x450")
root.configure(bg="#6caccf")

style = ttk.Style()
style.theme_use("clam")

frame = Frame(root, bg="#6caccf", padx=20, pady=20)
frame.pack(fill=BOTH, expand=True)

Label(frame, text="Enter Text", font=("Helvetica", 12, "bold"), bg="#6caccf").pack(anchor=W)
text_input = Text(frame, height=5, font=("Arial", 11),bg="#eff0f1")
text_input.pack(fill=X, pady=(5, 10))

Label(frame, text="Select Language", font=("Helvetica", 12, "bold"), bg="#6caccf").pack(anchor=W)
lang_choice = StringVar(root)
lang_choice.set("japanese")
lang_menu = ttk.Combobox(frame, textvariable=lang_choice, values=list(LANGUAGES.values()), font=("Arial", 11))
lang_menu.pack(fill=X, pady=(5, 15))

ttk.Button(frame, text="Translate", command=translate_text).pack(pady=5)

Label(frame, text="Translated Text", font=("Helvetica", 12, "bold"), bg="#6caccf").pack(anchor=W, pady=(15, 5))

text_output = Text(frame, height=7, font=("Arial", 11), bg="#eff0f1")
text_output.pack(fill=X)
root.mainloop()